## Steps til at køre scriptet:
1. Tag en kopi af denne notebook INDEN ændringer laves
2. Skriv dit navn (martin/pratt/charlotte) i en string med små bogstaver i variablen nedenfor

In [1]:
#eksempel:
name = "martin" #skal enten være: martin/pratt/charlotte
img_path = "/Users/MartinJohnsen/Documents/Martin Johnsen/MMC/3. Semester/Deep Learning/Projects/Algorithmic fairness/Data/celebA_resize3/"
run = "run1_"

In [2]:
from __future__ import print_function
#%matplotlib inline
import argparse
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import pandas as pd
import data_utils.data_utils_celeba_pytorch5 as data_utils
import data_utils.network_tuning23 as network_tuning
from IPython.display import clear_output
from skimage.io import imread
from skimage.transform import resize
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import pandas as pd
import pickle
# Load functions
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
from torch.nn import Linear, GRU, Conv2d, Dropout2d, MaxPool2d, BatchNorm2d
from torch.nn.functional import relu, elu, relu6, sigmoid, tanh, softmax

In [3]:
IMAGE_SHAPE = [73,60,3]
# Paths to data
# Root directory for dataset
if name=="charlotte":
    dataroot = '..\\AlgorithmicFairness'
    TRAIN_PATH =  dataroot + "\\Data\\train.csv" 
    VALID_PATH = dataroot + "\\Data\\valid.csv" 
    TEST_PATH = dataroot + "\\Data\\test.csv" 
    IMAGE_PATHS = "C:\\Users\\cfthe\\OneDrive\\DTU\\Kandidat\\Deep\\celebA_resize3\\"

else:
    dataroot = '../AlgorithmicFairness'
    TRAIN_PATH =  dataroot + "/Data/train.csv" 
    VALID_PATH = dataroot + "/Data/valid.csv" 
    TEST_PATH = dataroot + "/Data/test.csv" 
    IMAGE_PATHS = img_path
TARGET_COL = 'Smiling'
NUM_CLASSES = 2

# train holds both X (input) and t (target/truth)
data_train = data_utils.CelebADataset(TRAIN_PATH,IMAGE_PATHS,IMAGE_SHAPE,TARGET_COL)
data_valid = data_utils.CelebADataset(VALID_PATH,IMAGE_PATHS,IMAGE_SHAPE,TARGET_COL)

#tuning the network round 1
df = pd.DataFrame(index=list(range(1,11)), columns=['layers','activations','conv_out_channels','kernel_size','conv_stride','maxpool'\
                                                    ,'dropout','batchnorm','optimizer','learning_rate','weight_decay','batch_size','num_epochs'\
                                                    ,'net','train_loss','train_accs','valid_loss','valid_accs'])

df['layers'] = 1

#adjust activation function
for i in range(1,5):
    df.at[i,'activations'] = [relu, relu, relu, relu]
for i in range(5,11):
    df.at[i,'activations'] = [tanh, tanh, tanh, tanh, tanh, tanh]

#inserting combination of activations:
df.iloc[0,1][1]= tanh
df.iloc[1,1][1]= tanh
df.iloc[6,1][1]= relu
df.iloc[7,1][1]= relu

IMAGE_SHAPE = [73,60,3]
df['conv_out_channels'] = 16
df['kernel_size'] = 5
df['conv_stride'] = 1
df['maxpool'] = 1
df['dropout'] = 0.0
df['batchnorm'] = False
df['optimizer']='Adam'
df['learning_rate']=0.001
df['weight_decay']=0.0
df['batch_size']=128
df['num_epochs']=5

#adjust depth
df.at[1:4,'layers'] = 2
df.at[7:11,'layers'] = 2

#adjust channels
df.at[1,'conv_out_channels'] = 32 
df.at[2,'conv_out_channels'] = 64
df.at[3,'conv_out_channels'] = 32
df.at[4,'conv_out_channels'] = 64
df.at[5,'conv_out_channels'] = 32
df.at[6,'conv_out_channels'] = 128 
df.at[7,'conv_out_channels'] = 32
df.at[8,'conv_out_channels'] = 64
df.at[9,'conv_out_channels'] = 32
df.at[10,'conv_out_channels'] = 64 

In [4]:
if name == "martin":
    df = df.iloc[-3:11,:]
elif name == "pratt":
    df = df.iloc[0:3,:]
elif name == "charlotte":
    df = df.iloc[3:-3,:]

In [5]:
for i in list(df.index):#range(len(df)):
    print('Model: ', i)
    layers = df.loc[i,'layers']
    activations = df.loc[i,'activations']
    conv_out_channels = df.loc[i,'conv_out_channels']
    kernel_size = df.loc[i,'kernel_size']
    conv_stride = df.loc[i,'conv_stride']
    maxpool = int(df.loc[i,'maxpool'])
    dropout = df.loc[i,'dropout']
    batchnorm = df.loc[i,'batchnorm']
    optimizer = df.loc[i,'optimizer']
    learning_rate = df.loc[i,'learning_rate']
    weight_decay = df.loc[i,'weight_decay']
    batch_size = int(df.loc[i,'batch_size'])
    num_epochs = df.loc[i,'num_epochs']
    
    net = network_tuning.tune_architecture(layers, activations, IMAGE_SHAPE, conv_out_channels, kernel_size,conv_stride, maxpool, dropout, batchnorm)
    net_trained, df.at[i,'train_loss'], df.at[i,'train_accs'] = network_tuning.tune_train(net, data_train, optimizer, learning_rate, weight_decay, batch_size, num_epochs)
    df.at[i,'valid_loss'], df.at[i,'valid_accs'] = network_tuning.tune_valid(net, data_valid, batch_size)
    df.at[i,'net'] = run+name+'_model'+str(i)
    df.to_pickle(run+name+'_df.pkl')
    torch.save(net_trained.state_dict(), run+name+'_model'+str(i))

Model:  8
No GPU available.
Epoch:  0


/Users/MartinJohnsen/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:1339: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Epoch:  1
Epoch:  2


KeyboardInterrupt: 

In [ ]:
#plt.plot(pd.Series(df.loc[4,'train_accs']).rolling(window=600).mean())

In [ ]:
#pd.Series(df.loc[1,'train_accs']).rolling(window=600).mean()

In [1]:
import pickle

In [2]:
pwd

'/Users/MartinJohnsen/Documents/Martin Johnsen/MMC/3. Semester/Deep Learning/Projects/Algorithmic fairness/AlgorithmicFairness'

In [3]:
path = "/Users/MartinJohnsen/Documents/Martin Johnsen/MMC/3. Semester/Deep Learning/Projects/Algorithmic fairness/AlgorithmicFairness"

In [6]:
with open(path+'/aws/Models/run1_martin_df.pkl', 'rb') as f:
    data1 = pickle.load(f)

In [7]:
data1

,layers,activations,conv_out_channels,kernel_size,conv_stride,maxpool,dropout,batchnorm,optimizer,learning_rate,weight_decay,batch_size,num_epochs,net,train_loss,train_accs,valid_train_loss,valid_train_accs,valid_loss,valid_accs
1,2,"[<function relu at 0x12d1f5b70>, <function tan...",32,5,1,1,0.0,False,Adam,0.001,0.0,128,5,run1_martin_model1,"[0.7146734595298767, 5.96096658706665, 6.03815...","[0.5078125, 0.546875, 0.515625, 0.7421875, 0.6...","[0.8009383082389832, 0.5955037474632263, 0.384...","[0.4842703938484192, 0.8434086441993713, 0.870...",0.212693,0.910807
2,2,"[<function relu at 0x12d1f5b70>, <function tan...",64,5,1,1,0.0,False,Adam,0.001,0.0,128,5,run1_martin_model2,"[0.6914783716201782, 15.70244026184082, 36.217...","[0.5078125, 0.5625, 0.5, 0.4453125, 0.6328125,...","[0.7138357758522034, 1.0319395065307617, 0.725...","[0.49061256647109985, 0.8488951325416565, 0.86...",0.227125,0.907334
3,2,"[<function relu at 0x12d1f5b70>, <function rel...",32,5,1,1,0.0,False,Adam,0.001,0.0,128,5,run1_martin_model3,"[0.6933189034461975, 17.534706115722656, 11.21...","[0.5390625, 0.5625, 0.46875, 0.5703125, 0.5, 0...","[3.6073431968688965, 0.22339202463626862, 0.21...","[0.4773241877555847, 0.9045653343200684, 0.908...",0.231187,0.906679
4,2,"[<function relu at 0x12d1f5b70>, <function rel...",64,5,1,1,0.0,False,Adam,0.001,0.0,128,5,run1_martin_model4,"[0.7271662950515747, 93.59306335449219, 17.855...","[0.453125, 0.4921875, 0.59375, 0.46875, 0.4921...","[7.938321590423584, 0.2576132118701935, 0.2166...","[0.5079277157783508, 0.8980721831321716, 0.909...",0.228668,0.907787
5,1,"[<function tanh at 0x12d1f4620>, <function tan...",32,5,1,1,0.0,False,Adam,0.001,0.0,128,5,run1_martin_model5,"[0.7636500597000122, 6.108983516693115, 3.4058...","[0.5234375, 0.53125, 0.609375, 0.5, 0.671875, ...","[0.8202677369117737, 2.509509801864624, 1.4084...","[0.4853777587413788, 0.8198519945144653, 0.858...",0.2698,0.885841
6,1,"[<function tanh at 0x12d1f4620>, <function tan...",128,5,1,1,0.0,False,Adam,0.001,0.0,128,5,run1_martin_model6,"[0.8382572531700134, 13.698846817016602, 19.70...","[0.46875, 0.640625, 0.5625, 0.5546875, 0.67968...","[0.7192241549491882, 8.831503868103027, 11.681...","[0.5089847445487976, 0.7916142344474792, 0.842...",0.260439,0.896713
7,2,"[<function tanh at 0x12d1f4620>, <function rel...",32,5,1,1,0.0,False,Adam,0.001,0.0,128,5,run1_martin_model7,"[0.7210633158683777, 27.723573684692383, 1.008...","[0.4375, 0.46875, 0.640625, 0.5, 0.4453125, 0....","[0.714911699295044, 0.268008291721344, 0.29748...","[0.4878945052623749, 0.8858911991119385, 0.872...",0.248401,0.8947
8,2,"[<function tanh at 0x12d1f4620>, <function rel...",64,5,1,1,0.0,False,Adam,0.001,0.0,128,5,run1_martin_model8,"[0.7142513394355774, 54.387603759765625, 2.152...","[0.4921875, 0.4609375, 0.640625, 0.59375, 0.53...","[0.7034733295440674, 0.3434240520000458, 0.240...","[0.516383945941925, 0.8509588837623596, 0.8995...",0.241064,0.898374
9,2,"[<function tanh at 0x12d1f4620>, <function tan...",32,5,1,1,0.0,False,Adam,0.001,0.0,128,5,run1_martin_model9,"[0.7225079536437988, 5.823354244232178, 1.4886...","[0.4453125, 0.5625, 0.6875, 0.640625, 0.6875, ...","[0.7015874981880188, 0.5521948933601379, 0.692...","[0.49398499727249146, 0.833543062210083, 0.868...",0.529673,0.881009
10,2,"[<function tanh at 0x12d1f4620>, <function tan...",64,5,1,1,0.0,False,Adam,0.001,0.0,128,5,run1_martin_model10,"[0.7297398447990417, 8.323760032653809, 8.3655...","[0.453125, 0.53125, 0.640625, 0.65625, 0.57812...","[0.7187111377716064, 1.0636043548583984, 1.320...","[0.4739517867565155, 0.8660593032836914, 0.849...",0.814015,0.874566


In [9]:
list(data1.valid_accs)

[0.9108068346977234,
 0.9073337912559509,
 0.9066793918609619,
 0.9077867865562439,
 0.8858408331871033,
 0.896713137626648,
 0.89469975233078,
 0.8983741998672485,
 0.8810086846351624,
 0.874565839767456]

In [4]:
with open(path+'/aws/Models/run2__df.pkl', 'rb') as f:
    data = pickle.load(f)

In [5]:
data

,layers,activations,conv_out_channels,kernel_size,conv_stride,maxpool,dropout,batchnorm,optimizer,learning_rate,weight_decay,batch_size,num_epochs,net,train_loss,train_accs,valid_train_loss,valid_train_accs,valid_loss,valid_accs
1,2,"[<function relu at 0x12d1f5b70>, <built-in met...",32,5,1,2,0.2,True,Adam,0.001,0.01,128,5,run2__model1,"[0.787320077419281, 1.2756198644638062, 0.6986...","[0.3984375, 0.546875, 0.7109375, 0.671875, 0.6...","[0.7255932688713074, 0.27513042092323303, 0.24...","[0.5130618810653687, 0.8896159529685974, 0.895...",0.207665,0.910455
2,2,"[<function relu at 0x12d1f5b70>, <function rel...",32,5,1,2,0.2,True,Adam,0.001,0.01,128,5,run2__model2,"[0.7270410656929016, 1.192880392074585, 0.9200...","[0.46875, 0.546875, 0.609375, 0.6171875, 0.710...","[5.199552536010742, 0.27074000239372253, 0.254...","[0.5176926255226135, 0.8867971897125244, 0.894...",0.199299,0.91574
3,2,"[<function relu at 0x12d1f5b70>, <function rel...",64,5,1,2,0.8,True,Adam,0.001,0.10,128,5,run2__model3,"[0.8246510624885559, 1.9678641557693481, 1.537...","[0.4140625, 0.5078125, 0.53125, 0.609375, 0.59...","[3.5466063022613525, 0.37123364210128784, 0.27...","[0.4940856695175171, 0.8414456248283386, 0.881...",0.291153,0.89017
4,2,"[<function relu at 0x12d1f5b70>, <function rel...",32,5,1,2,0.2,True,Adam,0.001,0.01,128,5,run2__model4,"[0.852070152759552, 0.7770803570747375, 0.7371...","[0.390625, 0.6328125, 0.6484375, 0.6171875, 0....","[2.5280070304870605, 0.27008363604545593, 0.23...","[0.46172043681144714, 0.8894146084785461, 0.89...",0.208755,0.913626
5,3,"[<function relu at 0x12d1f5b70>, <function rel...",32,5,1,2,0.2,True,Adam,0.001,0.01,128,5,run2__model5,"[0.8077120780944824, 1.0716813802719116, 1.078...","[0.453125, 0.6484375, 0.6171875, 0.609375, 0.7...","[0.834692120552063, 0.2967720031738281, 0.2535...","[0.515125572681427, 0.8718478083610535, 0.8921...",0.204349,0.914632
6,4,"[<function relu at 0x12d1f5b70>, <function rel...",16,5,1,2,0.2,True,Adam,0.001,0.01,128,5,run2__model6,"[0.7874405384063721, 0.8448547720909119, 0.667...","[0.4375, 0.5859375, 0.6484375, 0.640625, 0.648...","[0.7212352156639099, 0.2527103126049042, 0.226...","[0.48104897141456604, 0.8892635703086853, 0.90...",0.22052,0.906075
7,3,"[<function relu at 0x12d1f5b70>, <function rel...",32,5,1,2,0.2,True,Adam,0.001,0.01,128,5,run2__model7,"[0.7538514733314514, 1.1628611087799072, 1.195...","[0.484375, 0.65625, 0.5859375, 0.65625, 0.6328...","[1.578385353088379, 0.25720539689064026, 0.262...","[0.479337602853775, 0.8903709650039673, 0.8862...",0.211087,0.912267
8,4,"[<function relu at 0x12d1f5b70>, <function rel...",16,5,1,2,0.2,True,Adam,0.001,0.01,128,5,run2__model8,"[0.7434457540512085, 1.2706197500228882, 0.632...","[0.5859375, 0.5078125, 0.7109375, 0.6640625, 0...","[2.1032886505126953, 0.23763082921504974, 0.23...","[0.48497506976127625, 0.8982231616973877, 0.89...",0.206461,0.911864


In [11]:
list(data.valid_accs)

[0.910454511642456,
 0.9157396554946899,
 0.890169620513916,
 0.9136255979537964,
 0.9146323204040527,
 0.9060754179954529,
 0.9122665524482727,
 0.9118638634681702]